In [12]:
!pip install openvino
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 52.3 MB/s eta 0:00:00


In [9]:
import openvino
import onnx
import numpy as np

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim

# Define a simple CNN architecture
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Model, loss, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

print('Finished Training')


100%|██████████| 9912422/9912422 [00:00<00:00, 239798078.44it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 24817802.46it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 65691705.74it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 13834806.66it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1, Loss: 0.8646281984712142
Epoch 2, Loss: 0.16114183335201637
Epoch 3, Loss: 0.10924505927899816
Epoch 4, Loss: 0.08574583027750127
Epoch 5, Loss: 0.07193660464679906
Epoch 6, Loss: 0.06300862835668551
Epoch 7, Loss: 0.05665075538286776
Epoch 8, Loss: 0.05119204524133021
Epoch 9, Loss: 0.04698519132669463
Epoch 10, Loss: 0.04307086843434276
Finished Training


In [34]:
# Continue from the previous PyTorch example

# Load the test dataset
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Function to test the model
def test_model(model, testloader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No need to track gradients
        for data in testloader:
            images, labels = data
            outputs = model(images)
            #print(np.shape(outputs))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model
accuracy = test_model(model, testloader)
print(f'Accuracy of the model on the 10000 test images: {accuracy} %')


AttributeError: 'Model' object has no attribute 'eval'

In [13]:
import copy
import torch.onnx


# Create a dummy input tensor matching the model's expected input size
# For example, a single 1x28x28 grayscale image
dummy_input = torch.randn(1, 1, 28, 28)

# Export the model
modelo = copy.deepcopy(model)
torch.onnx.export(model, dummy_input, "simple_model.onnx", export_params=True,
                          opset_version=11, do_constant_folding=True,
                          input_names=['input'], output_names=['output'],
                          dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
print("Model has been converted to ONNX format and saved as simple_model.onnx")



Model has been converted to ONNX format and saved as simple_model.onnx


In [33]:
from openvino.runtime import Core

# Initialize OpenVINO runtime
ie = Core()

# Read the network and weights
model_path = "simple_model.onnx"  # Update the path to your ONNX model
model = ie.read_model(model=model_path)

# Compile the model for a specific device
compiled_model = ie.compile_model(model=model, device_name="CPU")

# Create an infer request
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

correct = 0
total = 0

# Inference
for images, labels in testloader:
    images = images.numpy()  # Convert images to a numpy array

    for i in range(images.shape[0]):
        # Prepare the input, assuming the model expects images of size 28x28 pixels
        image = images[i]
        image = image.reshape((1, 1, 28, 28))  # Adjust shape to match the model's expected input

        # Perform inference
        results = compiled_model([image])[output_layer]


        # Get the index of the max log-probability
        pred = np.argmax(results, axis=1)
        correct += (pred == labels[i].numpy()).sum().item()
        total += labels.size(0)


print(f'Accuracy on the MNIST test set: {accuracy:.4f}')


Accuracy on the MNIST test set: 98.5800


In [19]:
from openvino.runtime import Core

def test_openvino_model(onnx_model_path, testloader):
    # Initialize OpenVINO inference engine
    ie = Core()
    model_path = "simple_model.onnx"
    model = ie.read_model(model=model_path)
    exec_net = ie.load_network(network=model, device_name='CPU')

    input_blob = next(iter(exec_net.input_info))
    out_blob = next(iter(exec_net.outputs))

    correct = 0
    total = 0

    for data in testloader:
        images, labels = data
        images = images.numpy()  # Convert PyTorch tensor to NumPy array

        for i in range(images.shape[0]):
            # Ensure input image is correctly shaped (batch size of 1 for individual predictions)
            image = images[i:i+1]

            # Perform inference
            result = exec_net.infer({input_blob: image})
            predictions = result[out_blob]

            # Get the index of the max log-probability
            predicted_class = np.argmax(predictions, axis=1)[0]

            # Compare with the ground truth
            correct += (predicted_class == labels[i].numpy()).item()

        total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy

# Assuming 'testloader' is your DataLoader for the MNIST test dataset
# and 'model_onnx' is the path to your ONNX model
model_onnx = 'path/to/your/model.onnx'
accuracy = test_openvino_model(model_onnx, testloader)
print(f'Accuracy of the model on the 10000 test images: {accuracy} %')


AttributeError: 'Core' object has no attribute 'load_network'

In [36]:
total

639232

In [37]:
correct

9868

In [18]:
import onnx

# Load the ONNX model
model = onnx.load("simple_model.onnx")

# Check that the model is well-formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
#onnx.helper.printable_graph(model.graph)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import numpy as np

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
        x = torch.relu(torch.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training settings
batch_size = 64
test_batch_size = 1000
epochs = 1  # Keeping epochs low for brevity; increase for real training

# Load data
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('../data', train=False, transform=transform)
train_loader = DataLoader(dataset1, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset2, batch_size=test_batch_size, shuffle=True)

# Instantiate model, loss, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Train the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)

# Test the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

test_start_time = time.time()
test(model, device, test_loader)
pytorch_test_time = time.time() - test_start_time


100%|██████████| 9912422/9912422 [00:00<00:00, 392842603.39it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 37329951.87it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 85935179.76it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 21405088.50it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw


Test set: Average loss: 0.0001, Accuracy: 9627/10000 (96%)



In [8]:
# Export the trained model to ONNX
dummy_input = torch.randn(1, 1, 28, 28, device=device)
torch.onnx.export(model, dummy_input, "simple_model.onnx", verbose=False)
from openvino.runtime import Core

# Initialize OpenVINO runtime
ie = Core()

model_path = "simple_model.onnx"  # Make sure the path is correct

# Read the model
model = ie.read_model(model=model_path)

# Compile the model for a specific device
compiled_model = ie.compile_model(model=model, device_name="CPU")

# Get names of the input and output layers
input_layer = next(iter(compiled_model.inputs))
output_layer = next(iter(compiled_model.outputs))

# Function to test the model using OpenVINO
def test_openvino(compiled_model, test_loader, input_layer, output_layer):
    correct = 0
    test_loss = 0
    criterion = nn.CrossEntropyLoss()

    start_time = time.time()
    for data, target in test_loader:
        # Preprocess the data and target
        data = data.np.astype(np.float32)
        target = target.np.astype(np.int64)

        # Perform inference
        result = compiled_model.create_infer_request().infer(inputs={input_layer.any_name: data})
        output = result[output_layer.any_name]

        # Calculate loss (use Torch for convenience)
        output_torch = torch.tensor(output)
        target_torch = torch.tensor(target)
        test_loss += criterion(output_torch, target_torch).item()

        # Calculate accuracy
        pred = np.argmax(output, axis=1)
        correct += np.sum(pred == target)

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    inference_time = time.time() - start_time

    print(f"\nOpenVINO Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n")
    return inference_time, accuracy

# Convert test_loader to a format suitable for OpenVINO inference (NumPy)
# Assuming the test_loader is not too large to fit into memory for simplicity
test_data = [(data.numpy(), target.numpy()) for data, target in test_loader]

# Measure OpenVINO model performance
openvino_test_time, openvino_accuracy = test_openvino(compiled_model, test_data, input_layer, output_layer)

print(f"PyTorch Inference Time: {pytorch_test_time:.3f} seconds")
print(f"OpenVINO Inference Time: {openvino_test_time:.3f} seconds")

# Note: Accuracy comparison assumes the same test set and that differences are due to computational precision
print(f"PyTorch Accuracy: {100. * correct / len(test_loader.dataset):.2f}%")
print(f"OpenVINO Accuracy: {openvino_accuracy:.2f}%")


AttributeError: 'Model' object has no attribute 'modules'

In [22]:
import time

In [28]:
ie = Core()

model_path = "simple_model.onnx"  # Make sure the path is correct

# Read the model
model = ie.read_model(model=model_path)

# Compile the model for a specific device
compiled_model = ie.compile_model(model=model, device_name="CPU")

# Get names of the input and output layers
input_layer = next(iter(compiled_model.inputs))
output_layer = next(iter(compiled_model.outputs))

# Function to test the model using OpenVINO
def test_openvino(compiled_model, test_loader, input_layer, output_layer):
    correct = 0
    test_loss = 0
    criterion = nn.CrossEntropyLoss()

    start_time = time.time()
    for data, target in test_loader:
        # Preprocess the data and target
        data = data.astype(np.float32)
        target = target.astype(np.int64)

        # Perform inference
        result = compiled_model.create_infer_request().infer(inputs={input_layer.any_name: data})
        output = result[output_layer.any_name]

        # Calculate loss (use Torch for convenience)
        output_torch = torch.tensor(output)
        target_torch = torch.tensor(target)
        test_loss += criterion(output_torch, target_torch).item()

        # Calculate accuracy
        pred = np.argmax(output, axis=1)
        correct += np.sum(pred == target)

    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader)
    inference_time = time.time() - start_time

    print(f"\nOpenVINO Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader)} ({accuracy:.0f}%)\n")
    return inference_time, accuracy

# Convert test_loader to a format suitable for OpenVINO inference (NumPy)
# Assuming the test_loader is not too large to fit into memory for simplicity
test_data = [(data.numpy(), target.numpy()) for data, target in testloader]

# Measure OpenVINO model performance
openvino_test_time, openvino_accuracy = test_openvino(compiled_model, testloader, input_layer, output_layer)

print(f"PyTorch Inference Time: {pytorch_test_time:.3f} seconds")
print(f"OpenVINO Inference Time: {openvino_test_time:.3f} seconds")

# Note: Accuracy comparison assumes the same test set and that differences are due to computational precision
print(f"PyTorch Accuracy: {100. * correct / len(test_loader.dataset):.2f}%")
print(f"OpenVINO Accuracy: {openvino_accuracy:.2f}%")

AttributeError: 'Tensor' object has no attribute 'astype'

In [30]:
ie = Core()

model_path = "simple_model.onnx"  # Make sure the path is correct

# Read the model
model = ie.read_model(model=model_path)

# Compile the model for a specific device
compiled_model = ie.compile_model(model=model, device_name="CPU")



# Define your data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Example normalization
])

# Load your dataset
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Initialize variables for accuracy calculation
correct = 0
total = 0

# Inference
for images, labels in testloader:
    images = images.numpy()  # Convert to NumPy array for OpenVINO
    for i in range(images.shape[0]):
        # Perform inference
        res = compiled_model.infer(inputs={next(iter(compiled_model.input_info)): images[i:i+1]})
        # Process results
        predictions = np.argmax(res[next(iter(compiled_model.outputs))], axis=1)
        correct += (predictions[0] == labels[i].numpy()).sum()
        total += 1

accuracy = correct / total
print(f'Accuracy: {accuracy:.2f}')


AttributeError: 'CompiledModel' object has no attribute 'infer'

In [31]:
correct = 0
total = 0

for data in testloader:
    images, labels = data
    # Convert images to NumPy array format expected by OpenVINO
    images = images.numpy()

    for i in range(images.shape[0]):
        input_blob = next(iter(exec_net.input_info))
        out_blob = next(iter(exec_net.outputs))
        # Note: Ensure the input image is correctly shaped for OpenVINO (N, C, H, W)
        image = images[i:i+1]  # Correctly maintain batch dimension

        result = exec_net.infer({input_blob: image})
        predictions = result[out_blob]
        predicted_class = np.argmax(predictions, axis=1)[0]

        # Compare with the ground truth
        correct += (predicted_class == labels[i].numpy()).item()

    # Increment total per batch, not per image
    total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy}')


NameError: name 'exec_net' is not defined